In [1]:
import json
import pandas as pd
import numpy as np
import re

### TASKS

Convert:
- Age to integer
- Market value strings to numeric
- Transfer fee strings to numeric
- Players nationalities to strings with 2 columns (nation 1, nation 2)

Add:
- Transfer type (loan, loan end, free, retire)

Store as JSON files per league

### DATA

In [8]:
def read_file(league, season, window):
    filename = 'data/{league}_{season}_{window}.json'.format(
        league=league,
        season=season,
        window=window
    )
    
    with open(filename, 'r', encoding="utf8") as f:
        data = json.load(f)

    res = []
    for team in data:
        in_ = pd.DataFrame(team['in'])
        if len(in_) > 0:
            in_.loc[:, 'dir'] = 'in'

        left = pd.DataFrame(team['left'])
        if len(left) > 0:
            left.loc[:, 'dir'] = 'left'

        team_df = pd.concat([in_, left])
        if len(team_df) > 0:
            team_df.loc[:, 'team_name'] = team['team']['team_name']
            team_df.loc[:, 'team_country'] = team['team']['team_country']
            team_df.loc[:, 'team_id'] = team['team']['team_id']
        
        res.append(team_df)
        
    res_df = pd.concat(res)
    res_df.loc[:, 'league'] = league
    res_df.loc[:, 'season'] = season
    res_df.loc[:, 'window'] = window
    
    return res_df

In [9]:
SCRAPE_LEAGUES = ['GB1', 'ES1', 'IT1', 'L1', 'FR1', 'PO1', 'NL1'] # GB1
SCRAPE_SEASONS = [2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
SCRAPE_WINDOWS = ["s", "w"]
#copiare parametri della funzione scrape_script
transfers_data = pd.DataFrame([])

for league in SCRAPE_LEAGUES:
    for season in SCRAPE_SEASONS:
        for window in SCRAPE_WINDOWS:
            print(league, season, window)
            df = read_file(league, season, window)
            transfers_data = pd.concat([
                transfers_data,
                df
            ])

GB1 2022 s
GB1 2022 w
GB1 2023 s
GB1 2023 w
GB1 2024 s
GB1 2024 w
ES1 2022 s
ES1 2022 w
ES1 2023 s
ES1 2023 w
ES1 2024 s
ES1 2024 w
IT1 2022 s
IT1 2022 w
IT1 2023 s
IT1 2023 w
IT1 2024 s
IT1 2024 w
L1 2022 s
L1 2022 w
L1 2023 s
L1 2023 w
L1 2024 s
L1 2024 w
FR1 2022 s
FR1 2022 w
FR1 2023 s
FR1 2023 w
FR1 2024 s
FR1 2024 w
PO1 2022 s
PO1 2022 w
PO1 2023 s
PO1 2023 w
PO1 2024 s
PO1 2024 w
NL1 2022 s
NL1 2022 w
NL1 2023 s
NL1 2023 w
NL1 2024 s
NL1 2024 w


In [10]:
transfers_data.dtypes

player_name             object
player_id               object
player_age              object
player_nat              object
player_pos              object
market_val              object
counter_team_country    object
counter_team_name       object
counter_team_id         object
transfer_fee            object
transfer_id             object
dir                     object
team_name               object
team_country            object
team_id                 object
league                  object
season                   int64
window                  object
dtype: object

In [ ]:
#transfers_data.head(3)

### PREPROCESS

In [11]:
# Age to integer
transfers_data.loc[:, 'player_age'] = pd.to_numeric(transfers_data.loc[:, 'player_age'], errors='coerce')

# Season to integer
transfers_data.loc[:, 'season'] = pd.to_numeric(transfers_data.loc[:, 'season'], errors='coerce', downcast='integer')

In [12]:
def parse_transfer_fee(transfer_fee_raw):
    amnt_pat = r"(€\d+\.*\d*)"
    amnt = re.findall(amnt_pat, transfer_fee_raw)
    
    if not amnt:
        return np.nan
    
    if 'Th.' in transfer_fee_raw:
        mult = 1000 
    elif 'm' in transfer_fee_raw:
        mult = 1000000
    else:
        mult = 1
    
    return float(amnt[0].replace("€", "")) * mult

In [13]:
transfers_data.loc[:, 'transfer_fee_amnt'] = transfers_data.loc[:, 'transfer_fee'].apply(parse_transfer_fee)

transfers_data.loc[:, 'is_free'] = transfers_data.loc[:, 'transfer_fee'].str.contains("free")

transfers_data.loc[
    transfers_data.is_free == True,
    'transfer_fee_amnt'
] = 0

transfers_data.loc[:, 'is_loan'] = transfers_data.loc[:, 'transfer_fee'].str.lower().str.contains("loan")
transfers_data.loc[:, 'is_loan_end'] = transfers_data.loc[:, 'transfer_fee'].str.contains("End of loan")

transfers_data.loc[:, 'is_retired'] = transfers_data.counter_team_id.str.contains("Retired")

transfers_data.loc[:, 'market_val_amnt'] = transfers_data.loc[:, 'market_val'].apply(parse_transfer_fee)

In [14]:
def parse_nation(x):
    
    return pd.Series({
        'player_nation': x[0] if x else np.nan,
        'player_nation2': x[1] if len(x) > 1 else np.nan
    })

transfers_data = pd.concat([
    transfers_data,
    transfers_data.player_nat.apply(parse_nation)
], axis=1)

In [15]:
transfers_data.head()

,player_name,player_id,player_age,player_nat,player_pos,market_val,counter_team_country,counter_team_name,counter_team_id,transfer_fee,...,season,window,transfer_fee_amnt,is_free,is_loan,is_loan_end,is_retired,market_val_amnt,player_nation,player_nation2
0,Gabriel Jesus,363205,25,[Brazil],CF,€50.00m,England,Manchester City,281,€52.20m,...,2022,s,52200000.0,False,False,False,False,50000000.0,Brazil,NaN
1,Fábio Vieira,537598,22,[Portugal],AM,€25.00m,Portugal,FC Porto,720,€35.00m,...,2022,s,35000000.0,False,False,False,False,25000000.0,Portugal,NaN
2,Oleksandr Zinchenko,203853,25,[Ukraine],LB,€25.00m,England,Manchester City,281,€35.00m,...,2022,s,35000000.0,False,False,False,False,25000000.0,Ukraine,NaN
3,Matt Turner,425306,28,"[United States, Lithuania]",GK,€5.00m,United States,New England Revolution,626,€5.90m,...,2022,s,5900000.0,False,False,False,False,5000000.0,United States,Lithuania
4,Marquinhos,668268,19,[Brazil],RW,€7.00m,Brazil,São Paulo Futebol Clube,585,€3.50m,...,2022,s,3500000.0,False,False,False,False,7000000.0,Brazil,NaN


### STORE DATASET

In [16]:
dataset_cols = [
        'league',
        'season',
        'window',
        'team_id',
        'team_name',
        'team_country',
        'dir',
        'player_id',
        'player_name',
        'player_age',
        'player_nation',
        'player_nation2',
        'player_pos',
        'counter_team_id',
        'counter_team_name',
        'counter_team_country',
        'transfer_fee_amnt',
        'market_val_amnt',
        'is_free',
        'is_loan',
        'is_loan_end',
        'is_retired',
        'transfer_id',
]

transfers_dataset = transfers_data.loc[
    :,
    dataset_cols
]

Create Dataset Transfers, with last year update (2024)

In [19]:
transfers_dataset.to_csv('dataset/transfers_complete.csv', index=False)

Merge csv with latest data and csv with previous data 

In [21]:
df1 = pd.read_csv('dataset/transfers.csv')
df2 = pd.read_csv('dataset/transfers_2024.csv')
merged_df = pd.concat([df1, df2], ignore_index=True)
merged_df.to_csv('dataset/transfer_20092024.csv', index=False)
